In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/mulcamnlp2021/Mecab-ko-for-Google-Colab/

/content/drive/My Drive/mulcamnlp2021/Mecab-ko-for-Google-Colab


In [ ]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4 MB 229 kB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 448 kB 100.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-03 07:50:14--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c3:9b0a, 2406:da00:ff00::34cc:ea4a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=BCKuNotjHvpDGX1Xnh5eLsjgo4E%3D

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 4.0 MB/s 
     |████████████████████████████████| 170 kB 18.1 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 133 kB 18.3 MB/s 
     |████████████████████████████████| 138 kB 21.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=a02c650b8c3ad90823aa928f18568c12552964a6150f93568ad2c9e2ff490347
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7c3746412314880c7945ef7269d6e110a7708582f1a4ab68a6222c56893f80c1
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Attempting uninstall: urlli

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="dacon_climate_project")

wandb: Currently logged in as: dasom-oh (use `wandb login --relogin` to force relogin)


In [ ]:
pip install tensorflow-addons

     |████████████████████████████████| 679 kB 4.0 MB/s 


In [ ]:
!pip install transformers==3.3.0

     |████████████████████████████████| 1.1 MB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 15.7 MB/s 
     |████████████████████████████████| 3.0 MB 23.8 MB/s 
     |████████████████████████████████| 895 kB 39.1 MB/s 


In [ ]:
 pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
from tensorflow.python.keras.layers.core import Dropout
# from tensorflow.python.keras.backend import dtype, learning_phase
import tqdm
import tensorflow_addons as tfa
from konlpy.tag import Mecab

from sklearn.metrics import accuracy_score,f1_score,precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import *
from tensorflow.keras import Model, layers
from tensorflow.keras.layers import Input, MaxPool1D, GlobalAveragePooling1D, Dense, Embedding, Lambda, BatchNormalization
from tensorflow.keras.utils import Sequence
import math

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 3
VALID_SPLIT = 0.2
MAX_LEN=200
DATA_IN_PATH='/content/drive/My Drive/mulcamnlp2021/datain/'

In [ ]:
try:
    with open(DATA_IN_PATH+'train_input.npy', 'rb') as f1, open(DATA_IN_PATH+'label_input.npy', 'rb') as f2, open(DATA_IN_PATH+'test_input.npy', 'rb') as f3, open(DATA_IN_PATH+'data_configs.json', 'rb') as f4:
        inputs_on_train= np.load(f1, allow_pickle=True)
        labels= np.load(f2, allow_pickle=True)
        test_inputs= np.load(f3, allow_pickle=True)
        data_configs = json.load(f4)
except:

    try:
        with open(DATA_IN_PATH+'clean_train_text.npy', 'rb') as f1, open(DATA_IN_PATH+'clean_test_text.npy', 'rb') as f2, open(DATA_IN_PATH+'label_input.npy', 'rb') as f3:
            clean_train_text= np.load(f1, allow_pickle=True)
            clean_test_text= np.load(f2, allow_pickle=True)
            labels= np.load(f3, allow_pickle=True)
    except:
        train=pd.read_csv(DATA_IN_PATH+'train.csv')
        test=pd.read_csv(DATA_IN_PATH+'test.csv')

        temp_col='data'

        for col in train.columns:
            try:
                train[temp_col] = train[temp_col] + train[col].astype(str)
                test[temp_col] = test[temp_col] + test[col].astype(str)
            except KeyError:
                if col !='label':
                    train[temp_col] = train[col].astype(str)
                    test[temp_col] = test[col].astype(str)

        train[temp_col].fillna('NAN', inplace=True)
        test[temp_col].fillna('NAN', inplace=True)

        labels=train['label'].iloc[:100]

        train=train[[temp_col, 'label']].iloc[:100]
        test=test[[temp_col]].iloc[:100]

        from nltk.tokenize import word_tokenize
        from nltk.corpus import stopwords
        from nltk.stem import WordNetLemmatizer

        def preprocessing(text, tokenizer, remove_stopwords=False, stop_words_kr=[], stop_words_en=[]):
            kr=re.sub("[a-zA-Z\s]","", text)
            word_text=tokenizer.pos(kr)
            if remove_stopwords:
                word_text=[token[0] for token in word_text if not token[0] in stop_words_kr and token[1] not in ['SSO', 'SY', 'SC', 'SSC', 'SN']]

            eng=re.sub("[가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
            word_tokens = word_tokenize(eng, language='english')

            wl = WordNetLemmatizer()
            for word in word_tokens: 
                word = word.strip().lower()       
                if word not in stop_words_en:
                    if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                        word_text.append(wl.lemmatize(word))
            return word_text

        stop_words_en = set(stopwords.words('english'))
        clean_train_text=[]
        clean_test_text=[]

        mecab = Mecab()#210625 되는지 확인해보기

        import sys
        #시간이 많이 걸립니다.#base line 코드입니다.
        for text in tqdm.tqdm(train[temp_col]):#174,304건
            try:
                preprocessed = preprocessing(text, mecab)
                clean_train_text.append(preprocessed)
            except:
                clean_train_text.append([])

        for text in tqdm.tqdm(test[temp_col]):
            if type(text) == str:
                preprocessed = preprocessing(text, mecab)
                clean_test_text.append(preprocessed)
            else:
                clean_test_text.append([])

        del mecab
        del stop_words_en
        del test
        del train

        np.save(open(DATA_IN_PATH+'clean_train_text.npy', 'wb'), np.array(clean_train_text, dtype=object))
        np.save(open(DATA_IN_PATH+'clean_test_text.npy', 'wb'), np.array(clean_test_text, dtype=object))

    tokenizer=Tokenizer()#단어제한 없이 전부 사용 >> Unable to allocate 849. GiB for an array with shape (174304, 653541) and data type float64
    tokenizer.fit_on_texts(clean_train_text) # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.

    word_vocab=tokenizer.word_index
    data_configs={}
    data_configs['vocab']=word_vocab
    data_configs['vocab_size'] = len(word_vocab)+1
    json.dump(data_configs, open(DATA_IN_PATH+'data_configs.json', 'w', encoding='utf8'), ensure_ascii=False)


    train_sequences=tokenizer.texts_to_sequences(clean_train_text)
    del clean_train_text
    test_sequences=tokenizer.texts_to_sequences(clean_test_text)
    del clean_test_text


    inputs_on_train=pad_sequences(train_sequences, padding='post')
    del train_sequences
    test_inputs=pad_sequences(test_sequences, padding='post', maxlen=inputs_on_train.shape[-1])
    del test_sequences


    TRAIN_INPUT_DATA = 'train_input.npy'
    TRAIN_LABEL_INPUT_DATA = 'label_input.npy'
    TEST_INPUT_DATA = 'test_input.npy'
    #baseline 코드입니다.
    import os
    if not os.path.exists(DATA_IN_PATH):
        os.makedirs(DATA_IN_PATH)
    np.save(open(DATA_IN_PATH+TRAIN_INPUT_DATA, 'wb'), inputs_on_train)#학습데이터
    labels = pd.get_dummies(labels).to_numpy()
    np.save(open(DATA_IN_PATH+TRAIN_LABEL_INPUT_DATA, 'wb'), labels)#학습데이터
    np.save(open(DATA_IN_PATH+TEST_INPUT_DATA, 'wb'), test_inputs)#제출데이터



In [ ]:
maxlen = inputs_on_train.shape[1]

from sklearn.model_selection import train_test_split
train_validation_data_arr, test_data, train_validation_labels, test_labels = train_test_split(inputs_on_train, labels, random_state=42)
# 학습데이터에서 resampling을 하기전에 검증데이터를 분할합니다.
train, validation_data_arr, train_labels, validation_labels_arr = train_test_split(train_validation_data_arr, train_validation_labels, random_state=42)


In [ ]:
# resampling  기법 사용할때
#학습데이터 resampling
from sklearn.utils import resample
train_data_sampled = None
train_labels_sampled = None

unique_lbls = np.argmax(np.unique(train_labels, axis=0), axis=1)

from scipy.sparse import csr_matrix, vstack; 
for lbl in unique_lbls:
    # indexes = np.argmax(train_labels, axis=1) ==lbl
    # train_inputs_resampled, labels_resampled = train[indexes], train_labels[indexes]
    # resampling
    train_inputs_resampled, labels_resampled = resample(train[train_labels == lbl], train_labels[train_labels==lbl], replace=True, n_samples=5000, random_state=1234)
    if isinstance(train_data_sampled, csr_matrix):
        train_data_sampled = vstack([train_data_sampled, csr_matrix(train_inputs_resampled)])
    else:
        train_data_sampled = csr_matrix(train_inputs_resampled)

    if isinstance(train_labels_sampled, csr_matrix):
        train_labels_sampled = csr_matrix(vstack([train_labels_sampled, csr_matrix(labels_resampled)]))
    else:
        train_labels_sampled = csr_matrix(labels_resampled)

del train
del train_labels

sample_len = train_data_sampled.shape[0]
index = np.arange(sample_len)# label별로 순서대로 쌓여있어서 학습시 과적합 문제가 발생합니다.
np.random.shuffle(index)# 각 label 별 데이터를 섞습니다

train_inputs = train_data_sampled[index, :]
del train_data_sampled
labels = train_labels_sampled[index, :]
del train_labels_sampled


In [ ]:
vocab_size =data_configs['vocab_size']
embedding_dim = 300

In [ ]:
model_input = Input(shape=(train_inputs.shape[-1]))
x1 = Embedding(vocab_size, embedding_dim)(model_input)
x2 = GlobalAveragePooling1D()(x1)
x3 = Dense(64, kernel_initializer='glorot_normal', kernel_regularizer=tf.keras.regularizers.l2(3))(x2)
x4 = layers.Dropout(0.5)(x3)
outputs = Dense(validation_labels_arr.shape[-1], activation='softmax')(x4)
model = Model(inputs=model_input, outputs=outputs)
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2574)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 2574, 300)         215880600 
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                19264     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 46)                2990      
Total params: 215,902,854
Trainable params: 215,902,854
Non-trainable params: 0
_______________________________________________

In [ ]:
learning_rate = 0.001
lr_decay = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate, 
                                                        labels.shape[0]/10, 
                                                        decay_rate=0.5, 
                                                        staircase=True)

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
f1 = tfa.metrics.F1Score(num_classes=validation_labels_arr.shape[-1], average='macro', threshold=None)
metrics = [f1, tf.keras.metrics.CategoricalAccuracy()]

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_decay)

In [ ]:
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight('balanced'
#                                                 , np.argmax(np.unique(train_labels, axis=0), axis=1)
#                                                 , np.argmax(train_labels, axis=1).reshape(-1))
# resampling 기법 사용할때
class_weights = class_weight.compute_class_weight('balanced'
                                                , np.argmax(np.unique(labels.todense(), axis=0), axis=1)
                                                , np.argmax(labels.toarray(), axis=1).reshape(-1))

In [ ]:

# resampling 기법 사용할때
class CIFAR10Sequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        # return math.ceil(len(self.x) / self.batch_size)
        return math.ceil(self.x.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        indexes = np.random.choice(self.y.shape[0], self.batch_size, replace=True)
        # batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[indexes]
        batch_y = self.y[indexes]

        return (batch_x.todense(), batch_y.todense())

In [ ]:
MODEL_SAVE_FOLDER_PATH = '/content/drive/My Drive/mulcamnlp2021/models'
model_file_path = f'{MODEL_SAVE_FOLDER_PATH}/DACON-{{epoch:d}}-{{val_loss:.5f}}-{{val_f1_score:.5f}}.hdf5'
cb_model_check_point = ModelCheckpoint(filepath=model_file_path, monitor='val_f1_score', verbose=1, save_best_only=True)
cb_early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# history = model.fit(train, train_labels
#                     , epochs=100, batch_size=50
#                     , validation_data=(validation_data_arr, validation_labels_arr)
#                     , class_weight=dict(zip(np.argmax(np.unique(train_labels, axis=0), axis=1), class_weights))
#                     ,callbacks=[cb_model_check_point, cb_early_stopping]
#                     )

# resampling 기법 사용할때
# fit model
num_epochs = 100
history = model.fit(CIFAR10Sequence(x_set=train_inputs, y_set=labels, batch_size=258)
                    , epochs=num_epochs
                    , validation_data=(validation_data_arr, validation_labels_arr)
                    , class_weight=dict(zip(np.argmax(np.unique(labels.todense(), axis=0), axis=1), class_weights))
                    , callbacks=[cb_model_check_point, cb_early_stopping]
                    )

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/50
381/381 [==============================] - ETA: 0s - loss: 38.6466 - f1_score: 0.0051 - categorical_accuracy: 0.0146

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


381/381 [==============================] - 474s 1s/step - loss: 38.6466 - f1_score: 0.0051 - categorical_accuracy: 0.0146 - val_loss: 3.8075 - val_f1_score: 1.9600e-04 - val_categorical_accuracy: 0.0045

Epoch 00001: val_f1_score improved from inf to 0.00020, saving model to /content/drive/My Drive/mulcamnlp2021/models/DACON-1-3.80748-0.00020.hdf5
Epoch 2/50
381/381 [==============================] - ETA: 0s - loss: 3.8515 - f1_score: 0.0020 - categorical_accuracy: 0.0044

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


381/381 [==============================] - 487s 1s/step - loss: 3.8515 - f1_score: 0.0020 - categorical_accuracy: 0.0044 - val_loss: 3.8033 - val_f1_score: 3.3233e-05 - val_categorical_accuracy: 7.6495e-04

Epoch 00002: val_f1_score improved from 0.00020 to 0.00003, saving model to /content/drive/My Drive/mulcamnlp2021/models/DACON-2-3.80333-0.00003.hdf5
Epoch 3/50
381/381 [==============================] - ETA: 0s - loss: 3.8123 - f1_score: 6.9006e-04 - categorical_accuracy: 0.0026

...

381/381 [==============================] - 483s 1s/step - loss: 3.8789 - f1_score: 0.0011 - categorical_accuracy: 0.0015 - val_loss: 3.8389 - val_f1_score: 1.0640e-05 - val_categorical_accuracy: 2.4478e-04

Epoch 00010: val_f1_score improved from 0.00001 to 0.00001, saving model to /content/drive/My Drive/mulcamnlp2021/models/DACON-10-3.83895-0.00001.hdf5
Epoch 11/50
381/381 [==============================] - ETA: 0s - loss: 3.8434 - f1_score: 3.4016e-04 - categorical_accuracy: 8.8506e-04

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


381/381 [==============================] - 484s 1s/step - loss: 3.8434 - f1_score: 3.4016e-04 - categorical_accuracy: 8.8506e-04 - val_loss: 3.8328 - val_f1_score: 1.9946e-05 - val_categorical_accuracy: 4.5897e-04

Epoch 00011: val_f1_score did not improve from 0.00001
Epoch 12/50
381/381 [==============================] - ETA: 0s - loss: 3.7952 - f1_score: 9.8681e-04 - categorical_accuracy: 0.0011

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


381/381 [==============================] - 482s 1s/step - loss: 3.7952 - f1_score: 9.8681e-04 - categorical_accuracy: 0.0011 - val_loss: 3.8284 - val_f1_score: 9.3104e-06 - val_categorical_accuracy: 2.1419e-04

Epoch 00012: val_f1_score improved from 0.00001 to 0.00001, saving model to /content/drive/My Drive/mulcamnlp2021/models/DACON-12-3.82843-0.00001.hdf5
